# Airline PPC – EDA & Allocation

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv(r"/mnt/data/airline_ppc_portfolio/data/clean/ppc_clean.csv")
agg = pd.read_csv(r"/mnt/data/airline_ppc_portfolio/data/clean/summary_by_publisher_match_group.csv")

# Quick KPIs
totals = {"impressions": int(df["impressions"].sum()),
           "clicks": int(df["clicks"].sum()),
           "spend": float(df["click_charges"].sum()),
           "bookings": int(df["booking"].sum()),
           "revenue": float(df["revenue"].sum())}
ctr = totals["clicks"]/totals["impressions"]
cpc = totals["spend"]/totals["clicks"]
cvr = totals["bookings"]/totals["clicks"]
roas = totals["revenue"]/totals["spend"]
totals, ctr, cpc, cvr, roas


In [ ]:

# Visuals
plt.figure(figsize=(7,6))
plt.scatter(agg["spend"], agg["revenue"])
plt.title("Spend vs Revenue by Group")
plt.xlabel("Spend"); plt.ylabel("Revenue"); plt.tight_layout(); plt.show()


## Rule-based recommendations

In [ ]:

def safe_ratio(n,d): 
    import numpy as np
    return np.where(d>0, n/d, np.nan)

agg["ctr"] = safe_ratio(agg["clicks"], agg["impressions"])
agg["cvr"] = safe_ratio(agg["bookings"], agg["clicks"])
agg["cpc"] = safe_ratio(agg["spend"], agg["clicks"])
agg["cpa"] = safe_ratio(agg["spend"], agg["bookings"])
agg["roas"] = safe_ratio(agg["revenue"], agg["spend"])

def recommend_row(r):
    sufficient_clicks = r["clicks"] >= 100
    roas = r["roas"] if pd.notna(r["roas"]) else 0
    if sufficient_clicks and roas >= 1.2:
        return "INCREASE_SPEND_+20%"
    if sufficient_clicks and roas <= 0.8:
        return "DECREASE_SPEND_-20%"
    if (r["clicks"] >= 50) and (r["bookings"] == 0):
        return "PAUSE_TEST"
    if r["clicks"] < 20:
        return "HOLD_LOW_DATA"
    return "HOLD"

agg["recommendation"] = agg.apply(recommend_row, axis=1)
agg.sort_values(["recommendation","roas"], ascending=[True, False]).head(20)


## Heuristic budget reallocation (bounded 0.5×–1.5×)

In [ ]:

alloc = agg.copy()
alloc["current_spend"] = alloc["spend"].fillna(0.0)
total_budget = float(alloc["current_spend"].sum())
alloc["score"] = alloc["roas"].fillna(0.0) - 1.0
alpha = 0.5
alloc["proposed"] = alloc["current_spend"] * (1 + alpha * alloc["score"])
alloc["min"] = 0.5 * alloc["current_spend"]
alloc["max"] = 1.5 * alloc["current_spend"]
alloc["proposed"] = alloc[["proposed","min","max"]].max(axis=1)
alloc["proposed"] = pd.concat([alloc["proposed"], alloc["max"]], axis=1).min(axis=1)
scale = total_budget / alloc["proposed"].sum() if alloc["proposed"].sum() else 1.0
alloc["proposed"] *= scale
alloc[["publisher_name","match_type","keyword_group","current_spend","proposed","roas"]].head(20)
